In [ ]:
import glob
import pandas as pd
from pandas.io.json import json_normalize
from csv import writer
import ujson as jsonimport pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



In [ ]:
#Get the Slovenian data and put it in a 'Brexit_tweets_stance' folder
#Get Bt_list.txt from https://github.com/stcoats/DHH19_Hackathon/blob/master/Bt_list.txt
#Do the following in the command line: 
#pip install twarc
#You will need Twitter developer account for the following:
#twarc configure
#twarc hydrate Bt_list.txt > Brexit_slovenia.jsonl

#Now in Python, get the tweet texts from the Slovenian data 


path = '/home/cloud-user/Downloads/Brexit_tweets_stance/'
all_files = glob.glob(path + "/2*")
li = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, header=None, sep = "\t")
    li=li.append(df)

with open("/home/cloud-user/Downloads/Brexit_tweets_stance/Brexit_tweets_stance/Brexit_slovenia_outfile.csv","w") as outfile:
    output = writer(outfile)
    with open("/home/cloud-user/Downloads/Brexit_tweets_stance/Brexit_tweets_stance/Brexit_slovenia.jsonl") as f:
        for line in f:
            tweet = json.loads(line)
            try:
                row = (tweet["id"],tweet["user"]["screen_name"],tweet["full_text"])
                
                values = [value for value in row]
                output.writerow(values)
            except:
                continue

In [ ]:
li.columns = ["id","stance"]
df.columns = ["id","screen_name","text"]
#data frame of labelled Slovenian Brexit tweets
slo_pn = pd.merge(df,li, on = "id")

In [ ]:

#slo_pn = pd.read_msgpack("/home/cloud-user/Hackathon/slovenia_pos_neu_neg.msg")
#data frame of our tweets
dhh = pd.read_msgpack("/home/cloud-user/taito_wrk/DONOTREMOVE/Hackathon19/dhh_tweets_frame.msg")

dhh1 = dhh[dhh["lang"].notnull()]
dhh1["lang"]=[str(x) for x in dhh1["lang"]]
dhh2 = dhh1[dhh1["lang"]=="en"]
slo_pn_training = slo_pn.iloc[0:30000]

X_train, X_test, y_train, y_test = train_test_split(slo_pn_training['text'], slo_pn_training['stance'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(slo_pn_training['text']).toarray()
slo_pn_training['category_id'] = slo_pn_training['stance'].factorize()[0]
labels = slo_pn_training.category_id
clf = MultinomialNB().fit(X_train_tfidf, y_train)
dhh_no_rt = dhh2[~dhh2["text"].str.contains("RT @")]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [ ]:
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [ ]:
for i,x in dhh_no_rt.iloc[0:100].iterrows():
    print(x["text"], clf.predict(count_vect.transform([x["text"]])))